In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot

In [14]:
f = r'C:\Users\rites\Desktop\mudracircle\bks_raw\Parsing_testing\DENA\Dena\pattern1\dena01.pdf'
tables = camelot.read_pdf(f, flavor='stream', pages='all', edge_tol=500)
tables

<TableList n=9>

In [23]:
tables[1].df

,0,1,2,3,4,5
0,,Last N Transactions:,-NIL-,,Amount Type:\n-NIL-,
1,,Amount From:,-NIL-,,Amount To:\n-NIL-,
2,,Instrument ID From:,-NIL-,,Instrument ID To:\n-NIL-,
3,,Choose Statement:,-NIL-,,,
4,Transactions List - -VEENA DYEING (INR) - 07...,,,,,
5,Date,Instrument,Transaction Id,Transaction Remarks,Cr/Dr\nAmount,Balance (INR)
6,30/04/2019,ID,,,Dr.,"1,02,112.53"
7,,828961,DC93908,SELF .,"1,00,000.00",
8,30/04/2019,,,,Cr.,"2,02,112.53"
9,,,DC57494,BY AHORE SILK MILLS,"2,00,000.00",


In [204]:
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df[ df.isnull().sum(axis=1) < df.shape[1]-2].reset_index(drop=True)
# df.head()

In [205]:
drop_idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index]
print(drop_idx)
print(len(drop_idx))
if len(drop_idx)>1:
    for i in drop_idx[1:]:
        print(i)
        df.drop(df.index[i],inplace=True)

[2, 28]
2
28


In [206]:
drop_idx[1:]

[28]

In [207]:
df.head()

,0,1,2,3,4,5,6
0,Account Number,Branch Name,Nominee Registered,Opening Balance,Closing Balance,NaN,NaN
1,041310008584,MAROL,No,868.84,"5,753.16",NaN,NaN
2,Date,Narration,Chq/Ref No.,Debit,Credit,NaN,Balance
3,02-Apr-2019,TRTR/909133727351/01-04-2019 09:44:51/UPI,NaN,600.00,NaN,268.84,NaN
4,02-Apr-2019,TRTR/909109661860/01-04-2019 09:40:03/MBS,NaN,NaN,600.00,868.84,NaN


In [208]:
try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
    df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('transaction').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('remarks').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
        except:
            print("\nICICI Column Headers Missing"); pass

In [209]:
df.head()

2,Date,Narration,Chq/Ref No.,Debit,Credit,NaN,Balance
0,02-Apr-2019,TRTR/909133727351/01-04-2019 09:44:51/UPI,NaN,600.00,NaN,268.84,NaN
1,02-Apr-2019,TRTR/909109661860/01-04-2019 09:40:03/MBS,NaN,NaN,600.00,868.84,NaN
2,02-Apr-2019,TRTR/909114980713/01-04-2019 14:51:24/MBS,NaN,NaN,"1,500.00","2,368.84",NaN
3,02-Apr-2019,TRTR/909140535910/01-04-2019 16:36:15/UPI,NaN,NaN,60.00,"2,428.84",NaN
4,02-Apr-2019,TRTR/909143302187/01-04-2019 19:09:12/UPI,NaN,NaN,20.00,"2,448.84",NaN


In [210]:
try:
    idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('page total').any(),axis=1)==True].index][0]    
    df.drop(df.index[idx2:], inplace=True)
except:pass

try:
    idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('total transaction count').any(),axis=1)==True].index][0]    
    df.drop(df.index[idx2:], inplace=True)
except:pass

try:
    idx2 = [c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('total amount').any(),axis=1)==True].index][0]    
    df.drop(df.index[idx2:], inplace=True)
except:pass

In [211]:
df.head()

2,Date,Narration,Chq/Ref No.,Debit,Credit,NaN,Balance
0,02-Apr-2019,TRTR/909133727351/01-04-2019 09:44:51/UPI,NaN,600.00,NaN,268.84,NaN
1,02-Apr-2019,TRTR/909109661860/01-04-2019 09:40:03/MBS,NaN,NaN,600.00,868.84,NaN
2,02-Apr-2019,TRTR/909114980713/01-04-2019 14:51:24/MBS,NaN,NaN,"1,500.00","2,368.84",NaN
3,02-Apr-2019,TRTR/909140535910/01-04-2019 16:36:15/UPI,NaN,NaN,60.00,"2,428.84",NaN
4,02-Apr-2019,TRTR/909143302187/01-04-2019 19:09:12/UPI,NaN,NaN,20.00,"2,448.84",NaN


In [212]:
list1 = list(df.columns)
list2 = []
for i in list1:
    if isnan(i) == False:
        list2.append(i)
df = df.iloc[:,:len(list2)]        
df.columns = list2

In [213]:
df.head()

,Date,Narration,Chq/Ref No.,Debit,Credit,Balance
0,02-Apr-2019,TRTR/909133727351/01-04-2019 09:44:51/UPI,NaN,600.00,NaN,268.84
1,02-Apr-2019,TRTR/909109661860/01-04-2019 09:40:03/MBS,NaN,NaN,600.00,868.84
2,02-Apr-2019,TRTR/909114980713/01-04-2019 14:51:24/MBS,NaN,NaN,"1,500.00","2,368.84"
3,02-Apr-2019,TRTR/909140535910/01-04-2019 16:36:15/UPI,NaN,NaN,60.00,"2,428.84"
4,02-Apr-2019,TRTR/909143302187/01-04-2019 19:09:12/UPI,NaN,NaN,20.00,"2,448.84"


In [214]:
try:
    df = df.loc[:, df.columns.notnull()]
except:
    print("pass")
    pass

In [215]:
df.to_csv("test.csv", index=False)